In [70]:
import numpy as np

test_file_path = "../data/day15/test.txt"
file_path = "../data/day15/input.txt"

def load_data(file_path):
    with open(file_path) as file:
        size = len(file.readline().strip())
    return np.genfromtxt(file_path, delimiter=np.ones(size, dtype='int8'), dtype='int8')

In [105]:
def neighbours(array, position):
    index_inc = np.array([[0,1],[1,0],[-1,0],[0,-1]])
    positions = position + index_inc
    positions = positions[((positions >= (0,0)) & (positions < array.shape)).all(axis=1)]
    return tuple(map(tuple, positions))

In [73]:
class PriorityNodeList:
  def __init__(self, shape):
    self.unknown = {(x, y) : np.iinfo(np.int32).max
                 for x in range(shape[0]) for y in range(shape[1])}
    self.known = {}

  def __setitem__(self, node, value):
    self.unknown.pop(node)
    self.known[node] = value

  def __contains__(self, node):
    return node in self.unknown or node in self.known

  def __getitem__(self, node):
    return self.known[node] if node in self.known else np.iinfo(np.int32).max

  def __len__(self):
    return len(self.known) + len(self.unknown)

  def pop(self):
    min_value = np.iinfo(np.int32).max
    min_node = (0,0)
    for position, value in self.known.items():
      if value < min_value:
        min_node = position
        min_value = value
    return min_node, self.known.pop(min_node)


In [74]:
def shortest_path(start, end, weights):
    unvisited = PriorityNodeList(weights.shape)

    distances = {(x, y): np.iinfo(np.int32).max
                 for x in range(weights.shape[0]) for y in range(weights.shape[1])}
    unvisited[start] = 0
    current_position, current_distance = unvisited.pop()
    while current_position != end and len(unvisited) > 0:
        distances[current_position] = current_distance
        for position in neighbours(weights, current_position):
            if position in unvisited:
                new_distance = weights[position] + current_distance
                old_distance = unvisited[position]
                if new_distance < old_distance:
                    unvisited[position] = new_distance
        current_position, current_distance = unvisited.pop()
    if current_position == end:
        return current_distance
    else:
        return None


weights = load_data(file_path)
shortest_path((0, 0), (99, 99), weights)


698

Part 2

In [97]:
def expand(array):
  expanded = np.vstack([array + i for i in range(5)])
  expanded = np.hstack([expanded + i for i in range(5)])
  mask_over_9 = expanded > 9
  expanded[mask_over_9] %= 10
  expanded[mask_over_9] += 1
  return expanded

In [103]:
expanded_weights = expand(load_data(test_file_path))
assert shortest_path((0,0), tuple(np.array(expanded_weights.shape) -1), expanded_weights) == 315

In [106]:
expanded_weights = expand(load_data(file_path))

shortest_path((0,0), tuple(np.array(expanded_weights.shape) -1), expanded_weights)

3022